In [ ]:
import os
from glob import glob
import subprocess 
import json
from copy import copy

import networkx as nx

In [ ]:
ROOT = '../..'

In [ ]:
package_dirs = list(map(os.path.dirname, glob(os.path.join(ROOT, 'packages/pymedphys*/src/pymedphys*/__init__.py'))))
package_dirs

In [ ]:
internal_packages = [os.path.basename(directory) for directory in package_dirs]
subpackages = copy(internal_packages)
subpackages.remove('pymedphys')
subpackages

In [ ]:
dependencies = {} 
for directory in package_dirs: 
    package = os.path.basename(directory) 
    dependencies[package] = json.loads(subprocess.run(
        ["pydeps", directory, "--external"], stdout=subprocess.PIPE).stdout) 
    


In [ ]:
dependencies['pymedphys'] += subpackages
dependencies['pymedphys']

In [ ]:
all_modules = set()

for dependency_list in dependencies.values(): 
    for item in dependency_list: 
        all_modules.add(item) 


In [ ]:
all_conversion = { 
    module: module 
    for module in all_modules 
}   

In [ ]:
conversion = { 
    **all_conversion, 
    'attr': 'attrs', 
    'PIL': 'Pillow', 
    'Image': 'Pillow', 
    'mpl_toolkits': 'matplotlib', 
    'docker': None, 
    'collections': None, 
    'dateutil': None 
}   

In [ ]:
dependencies_set = { 
    package: {conversion[module] for module in dependency_list if conversion[module] is not None} 
    for package, dependency_list in dependencies.items() 
}
dependencies_set

In [ ]:
dag = nx.DiGraph()

for key, values in dependencies_set.items(): 
    dag.add_node(key) 
    dag.add_nodes_from(values) 
    edge_tuples = [ 
        (key, value) for value in values 
    ] 
    dag.add_edges_from(edge_tuples) 

assert nx.is_directed_acyclic_graph(dag)

In [ ]:
topological = list(nx.topological_sort(dag))
topological

In [ ]:
minimal_deps = {}

for package in topological[::-1]:
    if package in internal_packages:
        already_dependend_on = set()
        package_decendants = nx.descendants(dag, package)
        for dependency in package_decendants:
            if dependency in minimal_deps.keys():
                for already_in in nx.descendants(dag, dependency):
                    already_dependend_on.add(already_in)
        
        minimal_deps[package] = package_decendants.difference(already_dependend_on)
        
minimal_deps

In [ ]:
tree = {}
for package, dependencies in minimal_deps.items():
    external = [
        dependency for dependency in dependencies 
        if dependency not in internal_packages
    ]
    internal = [
        dependency for dependency in dependencies 
        if dependency in internal_packages
    ]
    tree[package] = {
        "external": external,
        "internal": internal
    }

tree

In [ ]:
dependency_json_filepath = os.path.join(ROOT, 'dependencies.json')

with open(dependency_json_filepath, 'r') as file:
    data = json.load(file)
    
data['tree'] = tree

with open(dependency_json_filepath, 'w') as file:
    json.dump(data, file, indent=2, sort_keys=True)